In [1]:
import pandas as pd
import numpy as np
from alibi.explainers import Counterfactual
from certa.explain import explain
from certa.utils import merge_sources
from models.utils import from_type


In [2]:
dataset = 'beers'
model_type = 'dm'
model = from_type(model_type)
model.load('models/' + model_type + '/' + dataset)

Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


In [3]:
def predict_fn(x):
    return model.predict(x)


In [4]:
datadir = 'datasets/' + dataset
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

In [5]:
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], [])
train_df = merge_sources(gt, 'ltable_', 'rtable_', lsource, rsource, ['label'], [])

In [6]:
idx = 27
rand_row = test_df.iloc[idx]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]
rand_row.head()

label                                                       0
ltable_ABV                                            10.00 %
ltable_Beer_Name            Tongue Buckler - Imperial Red Ale
ltable_Brew_Factory_Name        Ballast Point Brewing Company
ltable_Style                         American Amber / Red Ale
Name: 27, dtype: object

In [19]:
import dice_ml
d = dice_ml.Data(dataframe=train_df,
                 continuous_features=[],
                 outcome_name='label')


In [20]:
m = dice_ml.Model(model=model, backend='sklearn')
exp = dice_ml.Dice(d,m)


In [21]:
query_instance = rand_row.copy().to_dict()
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")


ValueError: ('Feature', 'l', 'not present in training data!')

In [16]:
shape = (1,) + train_df.shape[1:]
target_proba = 1.0
tol = 0.01 # want counterfactuals with p(class)>0.99
target_class = 'other' # any class other than 7 will do
max_iter = 1000
lam_init = 1e-1
max_lam_steps = 10
learning_rate_init = 0.1
feature_range = (train_df.min(),train_df.max())

In [17]:

cf = Counterfactual(predict_fn, shape=shape, target_proba=target_proba, tol=tol,
                    target_class=target_class, max_iter=max_iter, lam_init=lam_init,
                    max_lam_steps=max_lam_steps, learning_rate_init=learning_rate_init,
                    feature_range=feature_range)


ValueError: Length mismatch: Expected axis has 11 elements, new values have 8 elements